In [ ]:
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
--this can be any stage with your images in it (S3 bucket, Azure blob, etc) with directory enabled
create or replace stage my_image_stage
DIRECTORY = (ENABLE = TRUE);

In [ ]:
put_result = session.file.put("snow://notebook/AA_SANDBOX.PUBLIC.MARKETING_CREATIVE_REVIEW/images/greyson-joralemon-9IBqihqhuHc-unsplash.jpg","@MY_IMAGE_STAGE", auto_compress= False)
put_result[0].status

In [ ]:
put_results = session.file.put(
    local_file_name="Campaign Creative Review Notebook/images/",
    stage_location="@demo_stage",
    auto_compress=False,     # let Snowflake gzip for you if True
    overwrite=True
)

# 4️⃣ optional: check the outcome
for r in put_results:
    print(r.source, r.target, r.status)

In [ ]:
--create a table to hold reference to images (image file location and approval being the key ones)

create or replace TABLE CREATIVE_IMAGE_REVIEW (
	APPROVED_BY VARCHAR,
    APPROVED_TIME TIMESTAMP,
    IMG_FILE FILE,
	LAST_MODIFIED TIMESTAMP_TZ(3),
	MD5 VARCHAR,
	FILE_URL VARCHAR
);

In [ ]:
---this statement be run in batch or part of a pipeline.  It will 
insert into creative_image_review 
select null, null, to_file(file_url) img_file, LAST_MODIFIED, MD5, FILE_URL from directory(@my_image_stage);